# RAG Implementation with Quantized-Models

This notebook implements a Retrieval-Augmented Generation (RAG) system using:
- Milvus for vector storage and retrieval
- Sentence Transformer for embedding generation
- Gemma for text generation

## 1. Import Dependencies

In [ ]:
import time
import numpy as np
import pandas as pd
from llama_cpp import Llama
from bert_score import score
from sentence_transformers import util
from pymilvus.model.hybrid import BGEM3EmbeddingFunction
from transformers import logging as transformers_logging
from pymilvus import DataType, MilvusClient, WeightedRanker, RRFRanker, AnnSearchRequest

In [ ]:
# ignore warnings
transformers_logging.set_verbosity_error()

## 2. Load Data

In [ ]:
data_file_name_= 'endpoints_info_v4.csv'
df = pd.read_csv(data_file_name_)

slice_df = df[['question','answer','context']]

pure_contexts = slice_df['context'].tolist()
questions = slice_df['question'].tolist()
answers = slice_df['answer'].tolist()
qa_as_context = (slice_df['question'] + ' ' + slice_df['answer']).to_list()
contexts = []
for pure_context in pure_contexts:
    contexts.append(pure_context)

# for qa in qa_as_context:
#     contexts.append(qa)


del slice_df
del df
del pure_contexts
del qa_as_context

## 3. Configure Embedding Model

In [ ]:
computation_device = "cuda:0"
# Load embedding model
def load_embedding_model(model_name='bge-m3'):
    """Load and configure the sentence transformer model for embeddings"""
    embedding_model_path = f"./models/{model_name}"
    
    # Load model from local path
    embedding_model = BGEM3EmbeddingFunction(
        model_name=embedding_model_path,
        device=computation_device
    )
    
    return embedding_model

# Initialize models
embedding_model = load_embedding_model()

## 4. Set Up Vector Database

In [ ]:
def setup_client_vector_db():
    """Initialize Milvus and create or get collection"""
    vector_db_client = MilvusClient( 
    uri = "http://localhost:19530",
    user= "admin",
    password= "admin",
    db_name= "default"
    )
    return vector_db_client

def setup_schema_vector_db(vector_db_client):
    schema = vector_db_client.create_schema(
    auto_id=False,
    enable_dynamic_field=True
        )
    # Add fields to schema
    schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
    schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=20000)
    schema.add_field(field_name="sparse", datatype=DataType.SPARSE_FLOAT_VECTOR)
    schema.add_field(field_name="dense", datatype=DataType.FLOAT_VECTOR, dim=1024)
    return schema 

def setup_index_vector_db(vector_db_client):
    index_params = vector_db_client.prepare_index_params()
    # Add indexes
    index_params.add_index(
        field_name="dense",
        index_name="dense_index",
        index_type="AUTOINDEX",
        metric_type="IP"
        )

    index_params.add_index(
        field_name="sparse",
        index_name="sparse_index",
        index_type="SPARSE_INVERTED_INDEX",  # Index type for sparse vectors
        metric_type="IP",  # Currently, only IP (Inner Product) is supported for sparse vectors
        params={"drop_ratio_build": 0.2},  # The ratio of small vector values to be dropped during indexing
        )    
    return index_params 

def setup_collection_vector_db(vector_db_client,collection_name, schema,index_params):
    result_drop_collection = vector_db_client.drop_collection(collection_name)
    result_setup_collection_vector_db = vector_db_client.create_collection(
        collection_name=collection_name,
        schema=schema,
        index_params=index_params
        )    
    return result_setup_collection_vector_db 


def preview_documents_in_collection_vector_db(vector_db_client,collection_name,filter_expression = "text like '%%'"):
    result_preview_documents_in_collection_vector_db = vector_db_client.query(collection_name, filter = filter_expression, output_fields= ["text"])
    return result_preview_documents_in_collection_vector_db 


def delete_documents_in_collection_vector_db(vector_db_client,collection_name,filter_expression = "text like '%%'"):
    result_delete_documents_in_collection_vector_db = vector_db_client.delete(collection_name, filter = filter_expression)
    return result_delete_documents_in_collection_vector_db 



# Set up Milvus
collection_name = 'endpoints_info'
vector_db_client = setup_client_vector_db()
schema_vector_db = setup_schema_vector_db(vector_db_client)
index_params_vector_db = setup_index_vector_db(vector_db_client)
result_setup_collection_vector_db = setup_collection_vector_db(vector_db_client,collection_name, schema_vector_db,index_params_vector_db)

## 5. Add Documents to Vector Database

In [ ]:
def extract_sparse_vector(encoded_document):    
    # Convert to dictionary format {index: value}
    coo = encoded_document.tocoo()
    sparse_dict = {int(col): float(val) for col, val in zip(coo.col, coo.data)}
    return sparse_dict

def l2_normalize(vec):
    norm = np.linalg.norm(vec)
    return (vec / norm).tolist() if norm > 0 else vec

def add_documents_to_collection(vector_db_client, collection_name, documents, embedding_model):
    """Add documents to Milvus collection with embeddings"""
    # Generate embeddings for documents
    encoded_documents  = embedding_model.encode_documents(documents)
    print("Dense vector shape:", encoded_documents["dense"][0].shape)
    print("Sparse vector shape:", list(encoded_documents["sparse"])[0].shape)
    
    # Prepare id
    result_flush = vector_db_client.flush(collection_name)    
    row_count = vector_db_client.get_collection_stats(collection_name)['row_count']

    data = []
    for i in range(len(encoded_documents['dense'])):
        # Get the sparse vector for this row
        sparse_dict = extract_sparse_vector(encoded_documents['sparse'][i,:])

        # Get and normalize dense vector for this row
        dense_normalized_list = l2_normalize(encoded_documents['dense'][i].tolist())    

        data.append({
            "id": row_count + i,
            "text": documents[i],
            "sparse": sparse_dict,
            "dense": dense_normalized_list 
        })
    
    # Add documents with embeddings to collection
    result_add_documents_to_collection = vector_db_client.insert(
        collection_name=collection_name,
        data=data
    )
    return result_add_documents_to_collection['insert_count']

# Add documents to collection
insert_count = add_documents_to_collection(vector_db_client, collection_name, contexts, embedding_model)
print(f"Added {insert_count} documents to vector database")

In [ ]:
# delete_documents_in_collection_vector_db(vector_db_client,collection_name)

## 6. Implement Retrieval Function

In [ ]:
def prepare_ann_search_request(query_dense_vector, query_sparse_vector, top_k):
    search_param_1 = {
        "data": [query_dense_vector],
        "anns_field": "dense",
        "param": {
            "metric_type": "IP",
            "params": {"nprobe": 10}
        },
        "limit": top_k
    }
    request_1 = AnnSearchRequest(**search_param_1)

    search_param_2 = {
        "data": [query_sparse_vector],
        "anns_field": "sparse",
        "param": {
            "metric_type": "IP",
            "params": {"drop_ratio_build": 0.2}
        },
        "limit": top_k
    }
    request_2 = AnnSearchRequest(**search_param_2)

    reqs = [request_1, request_2]
    
    return reqs

def retrieve_relevant_documents(vector_db_client, query, collection_name, embedding_model, top_k=2, similarity_threshold=0.48):
    """Retrieve relevant documents based on semantic similarity"""
    # Create embedding for query
    query_embedding = embedding_model.encode_documents([query])
    query_dense_vector = l2_normalize(query_embedding['dense'][0].tolist())
    query_sparse_vector = extract_sparse_vector(query_embedding['sparse'][0,:])

    # Prepare ANN Search Request
    requests = prepare_ann_search_request(query_dense_vector, query_sparse_vector, top_k)

    # Setup Ranker
    ranker = WeightedRanker(0.3, 0.85) 

    # ranker = RRFRanker(100)

    # Query the collection
    results = vector_db_client.hybrid_search(
        collection_name=collection_name,
        reqs=requests,
        ranker=ranker,
        limit=top_k,
        output_fields=["text"],
    )


    not_found_message = ["برای پاسخ این سوال با ادمین تماس بگیرید."]
    not_found_message = [""]

    
    # Extract results
    results = results[0]
    documents = []
    distances = []
    for hits in results:
        documents.append(hits['entity']['text'])
        distances.append(hits['distance'])
    
    # Print similarity scores for debugging
    print(f"Similarity scores: {[d for d in distances]}")
    
    # Optional: Filter by similarity threshold
    filtered_docs = [doc for doc, dist in zip(documents, distances) if dist >= similarity_threshold]
    return filtered_docs if filtered_docs else not_found_message
    
    # return documents

# Test retrieval function
top_k = 2
test_query = questions[-14]
print(test_query)
retrieved_docs = retrieve_relevant_documents(vector_db_client, test_query, collection_name, embedding_model,top_k)
# print(retrieved_docs)
for retrieved_docs in retrieved_docs:
    print(f"Retrieved document: {retrieved_docs[:400]}...")

## 7. Evaluation Retreival System

In [ ]:
def evaluate_retrieval(retrieved_contexts, expected_context, top_k=top_k):
    """
    Checks at which rank the expected (gold) context was retrieved
    and calculates basic metrics like MRR (Mean reciprocal rank).
    """
    for rank, doc in enumerate(retrieved_contexts, start=1):  # ranks start at 1
        if expected_context in doc:
            return {
                "rank": rank,
                "recall@k": 1 if rank <= top_k else 0,
                "MMR": 1 / rank
            }
    
    # Not found
    return {
        "rank": None,
        "recall@k": 0,
        "MMR": 0
    }

In [ ]:
def generate_retrieval_system_report(vector_db_client, questions, expected_contexts, collection_name, top_k):
    reports = {
        'question': [],
        'expected_context': [],
        'retrieved_contexts': [],
        'rank': [],
        'recall@k': [],
        'MMR': [],
    }
    for question, expected_context in zip(questions,expected_contexts):
        retrieved_contexts = retrieve_relevant_documents(vector_db_client, question, collection_name, embedding_model, top_k)
        metrics = evaluate_retrieval(retrieved_contexts, expected_context, top_k=top_k)
        rank = metrics['rank']
        recall_at_k = metrics['recall@k']
        mmr = metrics['MMR']
        reports['question'].append(question)
        reports['expected_context'].append(expected_context)
        reports['retrieved_contexts'].append(retrieved_contexts)
        reports['rank'].append(rank)
        reports['recall@k'].append(recall_at_k)
        reports['MMR'].append(mmr)
        print("Question: ",question)
        print("Found: ",recall_at_k)
        print("Rank: ", rank)
        print("MMR: ", mmr)
        print("Retrieved Contexts count: ", len(retrieved_contexts))
        print("======================================")
        print("======================================")
    return reports

retrieval_system_report = generate_retrieval_system_report(vector_db_client, questions, contexts, collection_name, top_k)

In [ ]:
def save_retrieval_system_report(reports):
    df_reports = pd.DataFrame(reports)
    df_reports.to_csv('retrieval_system_report.csv', index=False)
    # print("Cosine Similarity Average:",df_reports['cosine'].mean(), '%')
    # print("Sum processing time:",df_reports['processing_time'].astype(float).sum(), 'sec')
    # print("Report saved to the file successfully.")
    return df_reports

df_retrieval_system_report = save_retrieval_system_report(retrieval_system_report)

In [ ]:
df_retrieval_system_report[df_retrieval_system_report['recall@k'] == 0]

## 8. Load LLM for Generation

In [ ]:
context_window_size = 28000

def load_llm_model(model_path="./models/gemma-3-4b-it.Q2_K/gemma-3-4b-it.Q2_K.Q8_0.gguf",chat_format='gemma'):
    """Load and configure the LLM for text generation"""
    llm = Llama(
        model_path=model_path,
        chat_format=chat_format,
        n_gpu_layers=-1,  # Use all available GPU layers
        n_ctx=context_window_size,       # Context window size
        verbose=False
    )
    
    return llm


# Define LLM models details like: path and chat_format
llm_models_details = {
    'dorna-llama3-8b-q8' : {'path': './models/Dorna-Llama3-8B-Instruct-GGUF-Q8/dorna-llama3-8b-instruct.Q8_0.gguf',
                            'chat_format': 'llama-3'},
    'deepseek-r1-7b-qwen' : {'path': './models/DeepSeek-R1-Distill-Qwen-7B-GGUF/DeepSeek-R1-Distill-Qwen-7B.Q8_0.gguf',
                            'chat_format': 'gemma'},
    'gemma-3-4b-q2': {'path':'./models/gemma-3-4b-it.Q2_K/gemma-3-4b-it.Q2_K.gguf',
                     'chat_format': 'gemma'},
    'gemma-3-4b-q8': {'path':'./models/gemma-3-4b-it.Q8_0/gemma-3-4b-it.Q8_0.gguf',
                      'chat_format': 'gemma'},
    'gemma-3-4b-fp16': {'path':'./models/gemma-3-4b-it.fp16/gemma-3-4b-it.fp16.gguf',
                        'chat_format': 'gemma'}
    }

# Load Llama model
target_llm_model = 'gemma-3-4b-q8'
llm_model_path, llm_chat_format = llm_models_details[target_llm_model]['path'], llm_models_details[target_llm_model]['chat_format']
llm = load_llm_model(llm_model_path, llm_chat_format)

## 9. Load NLP model for Summarization

In [ ]:

# def load_summarization_model(model_path="./models/mt5-persian-summary/"):
#     """Load and configure the MT5 model for Persian summarization"""    
#     # Initialize components
#     tokenizer = MT5Tokenizer.from_pretrained(
#         model_path,
#         local_files_only=True,
#         legacy=False,
#         use_fast=True
#     )
    
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     model = AutoModelForSeq2SeqLM.from_pretrained(
#         model_path,
#         local_files_only=True
#     ).to(device)
    
#     return model, tokenizer, device

# # Define available summarization models
# summarization_models = {
#     'mt5-persian-base': {
#         'path': './models/mt5-persian-summary/',
#         'description': 'Base MT5 model fine-tuned for Persian summarization'
#     },
#     'parst5-summary': {
#         'path': './models/parsT5-base/',
#         'description': 'Specialized Persian T5 for summarization'
#     }
# }

# # Load model
# target_model = 'mt5-persian-base'
# summarizer_model, summarizer_tokenizer, summarizer_device = load_summarization_model(
#     summarization_models[target_model]['path']
# )

## 10. Create RAG Pipeline

In [ ]:
# Define prompt template
PROMPT_TEMPLATE = '''
کانتکست:
{context}


تاریخچه مکالمات:
{history}


سوال کاربر:
{prompt}
'''


technical_support_system_message = """
<role>You are a Technical Assistant.</role>

<goal>
  Answer user technical questions concisely and accurately, using only the retrieved context and conversation history.
</goal>

<behavior>
  <rule>Always pay close attention to the conversation history.</rule>
  <rule>Only use retrieved context and conversation history; any external or additional information is strictly forbidden.</rule>
  <rule>Your response must be a single, very short sentence.</rule>
  <rule>Do not provide extra explanations, introductions, or commentary unless explicitly requested by the user (e.g., "Tell me more",”بیشتر توضیح میدی”, "بیشتر توضیح بده", "مثال بزن", or "جزییات بگو).</rule>
  <rule>Do not repeat the user's question in your answer.</rule>
  <rule>If the answer is not present in the context, reply only: «اطلاعاتی در دسترس نیست. با ادمین تماس بگیرید.»</rule>
  <rule>If this is the first question in the conversation, respond with just one short sentence.</rule>
  <rule>If the user asks a follow-up such as "Tell me more", "Give an example", or "Provide details", you may give a detailed, multi-sentence answer.</rule>
  <rule>All responses must be in Persian (Farsi).</rule>
</behavior>

<steps>
  <step0 title="Single-Sentence Response">
    <trigger>If user asks a new question or it is the first turn in the conversation.</trigger>
    <action>Reply with only one very short sentence in Persian, strictly based on retrieved context and conversation history.</action>
  </step0>

  <step1 title="Follow-Up (Details)">
    <trigger>If user requests more detail (e.g., says "Tell me more", "Give an example", "Provide details" ,"بیشتر توضیح بده", "مثال بزن", "جزییات بگو”, or ”بیشتر توضیح میدی”).</trigger>
    <action>Provide a detailed, multi-sentence answer in Persian, but only using retrieved context and conversation history.</action>
  </step1>

  <step2 title="No Answer Available">
    <trigger>If the answer is not present in the context or history.</trigger>
    <action>Reply: «اطلاعاتی در دسترس نیست. با ادمین تماس بگیرید.»</action>
  </step2>
</steps>

<style>
  <item>Responses must be very short, clear, and without unnecessary details.</item>
  <item>Use a formal and respectful tone in Persian.</item>
  <item>Use proper punctuation in Persian.</item>
  <item>Do not use emojis unless the user requests them.</item>
  <item>Technical terminology allowed</item>
  <item>Zero pleasantries</item>
  <item>Bullet points only when explicitly requested</item>
</style>

<guardrails>
  <rule>Never use information outside of retrieved context and conversation history.</rule>
  <rule>Never generate explanations or commentary unless explicitly requested by the user.</rule>
  <rule>Never repeat the user's question in your answer.</rule>
  <rule>Never provide more than one short sentence unless the user asks for more detail.</rule>
  <rule>All responses must be in Persian (Farsi).</rule>
</guardrails>

<end_tag>**ASSISTANT_READY**</end_tag>
"""



summarizer_system_message = """
تو یک خلاصه‌کننده متون هستی. وظیفه تو فقط و فقط خلاصه کردن است.
- هرگونه سلام و احوالپرسی را حذف کن
- فقط هسته اصلی متن را نگه دار
- اگر متن کوتاه است (کمتر از 10 کلمه)، عیناً تکرارش کن
- تحت هیچ شرایطی به متن پاسخ نده
- فقط متن خلاصه شده یا متن اصلی را برگردان
"""


# Initialize conversation history
conversation_history = []


def summarize_query_with_llm(query,llm=llm):
    """Summarize query using the LLM"""
    prefix = 'متن زیر را خلاصه کن:\n'
    query = prefix + query
    messages = [
        {"role": "system", "content": summarizer_system_message},
        {"role": "user", "content": "سلام دوست عزیز، می‌خواستم بپرسم معنی کد ۵۰۲ چیه؟"},
        {"role": "assistant", "content": "معنی کد ۵۰۲ چیه؟"},
        {"role": "user", "content": f"{query}"}
    ]
    
    response = llm.create_chat_completion(
        messages=messages,
        top_p=0.85,
        temperature=0.0  # Low temperature for more deterministic responses
    )
    # .split('</think>')[-1] if the model thinks!
    response_clean = response['choices'][0]['message']['content']
    print(f"Summarized version: {response_clean}")

    return response_clean
    


# def summarize_query_with_mt5(text, model, tokenizer, device, max_length=250):
#     """Generate summary using the loaded model"""
#     # Persian-specific task prefix
#     input_text = f"خلاصه‌سازی متن فارسی: {text}"
    
#     # Tokenize and move to correct device
#     inputs = tokenizer(
#         input_text,
#         return_tensors="pt",
#         max_length=512,
#         truncation=True,
#         padding="max_length"
#     ).to(device)
    
#     # Generate with optimized parameters
#     response = model.generate(
#         input_ids=inputs.input_ids,
#         attention_mask=inputs.attention_mask,
#         max_length=max_length,
#         num_beams=4,
#         repetition_penalty=2.5,
#         length_penalty=0.8,
#         early_stopping=True,
#         no_repeat_ngram_size=3
#     )
#     response_clean = tokenizer.decode(response[0], skip_special_tokens=True)
#     print(f"Summarized version: {response_clean}")
#     return response_clean



def retrieve_context(vector_db_client, query, collection_name=collection_name, embedding_model=embedding_model,top_k=top_k):
    """Retrieve relevant context based on the query"""
    docs = retrieve_relevant_documents(vector_db_client, query, collection_name, embedding_model,top_k)
    for doc in docs:
        print(f"Retrieved document: {doc[:100]}...")
    return "\n".join(docs)


def generate_response_stream(model_input, llm=llm):
    """Generate streaming response using the LLM"""
    messages = [
        {"role": "system", "content": technical_support_system_message},
        {"role": "user", "content": f"{model_input}"}
    ]
    
    response = llm.create_chat_completion(
        messages=messages,
        top_p=0.85,
        temperature=0.1,  # Low temperature for more deterministic responses
        repeat_penalty= 1.2,
        stream=True
    )
    
    full_response = ""
    for chunk in response:
        delta = chunk['choices'][0]['delta']
        if 'content' in delta:
            content = delta['content']
            print(content, end='', flush=True)
            full_response += content
            yield content


def generate_response(model_input, llm=llm):
    """Generate response using the LLM"""
    messages = [
        {"role": "system", "content": technical_support_system_message},
        {"role": "user", "content": f"{model_input}"}
    ]
    
    response = llm.create_chat_completion(
        messages=messages,
        top_p=0.85,
        temperature=0.1,  # Low temperature for more deterministic responses
      repeat_penalty= 1.2
    )
    # .split('</think>')[-1] if the model thinks!
    response_clean = response['choices'][0]['message']['content']
    print(response_clean)

    return response_clean


def rag_chat(user_query, history=None, stream=False, summary = False):
    """Complete RAG pipeline: Retrieve → Generate → Respond"""
    if history is None:
        history = conversation_history
    
    # Format conversation history
    history_text = "\n".join(history)

    if summary:
        # Summarize input query
        # With MT5
        # user_query_summarized = summarize_query_with_mt5(user_query, summarizer_model, summarizer_tokenizer, summarizer_device)
        # With LLM
        user_query_summarized = summarize_query_with_llm(user_query)
        
        # Retrieve relevant context
        context = retrieve_context(user_query_summarized)
    
    
    else:
        # Retrieve relevant context
        context = retrieve_context(vector_db_client, user_query, collection_name, embedding_model,top_k)
    
    
    # Create prompt with context and history
    prompt = PROMPT_TEMPLATE.format(
        history=history_text,
        context=context, 
        prompt=user_query
    )

    if stream:
        # Generate streaming response
        response = ""
        response_stream = generate_response_stream(prompt)
        for chunk in response_stream:
            response += chunk
    else:
        # Generate response (non-stream)
        response = generate_response(prompt)

    if summary:
        # Summarize ouput query 
        # With MT5
        # response_summarized = summarize_query_with_mt5(response, summarizer_model, summarizer_tokenizer, summarizer_device)
        # With LLM
        response_summarized = summarize_query_with_llm(response)
        
        # Update conversation history
        history.append(f"سوال کاربر: {user_query_summarized}")
        history.append(f"پاسخ سیستم: {response_summarized}")

    else:
        # Update conversation history
        history.append(user_query)
        history.append(response)
    
    llm.reset()
    return response,context

## 11. Test RAG System

In [ ]:
# # Example 1: Basic question (With Summary)
# query1 = questions[0]
# print(f"User query: {query1}")

# # Reset conversation history
# conversation_history = []

# # Time the response
# start = time.time()
# response, context = rag_chat(query1,stream = True, summary=True)
# end = time.time()

# print(f"---\nProcessing time: {end - start:.2f} seconds")

In [ ]:
# Example 1: Basic question (Without Summary)
query1 = questions[0]
print(f"User query: {query1}")

# Reset conversation history
conversation_history = []

# Time the response
start = time.time()
response, context = rag_chat(query1,stream = True, summary=False)
end = time.time()

print(f"---\nProcessing time: {end - start:.2f} seconds")

In [ ]:
# Example 2: Follow-up question
query2 = "میشه بیشتر راجع به این توضیح بدی؟"
print(f"User query: {query2}")

# Time the response (using existing conversation history)
start = time.time()
response, context = rag_chat(query2,stream = True)
end = time.time()

print(f"---\nProcessing time: {end - start:.2f} seconds")

In [ ]:
# Example 3: Different topic question
query3 = questions[-2]
print(f"User query: {query3}")

# Reset conversation history
conversation_history = []

# Time the response
start = time.time()
response, context = rag_chat(query3,stream = True)
end = time.time()


print(f"---\nProcessing time: {end - start:.2f} seconds")

In [ ]:
# Example 2: Follow-up question
print(f"User query: {query2}")

# Time the response (using existing conversation history)
start = time.time()
response, context = rag_chat(query2,stream = True)
end = time.time()

print(f"---\nProcessing time: {end - start:.2f} seconds")

## 11. RAG System Evaluation

Test with more complex queries to evaluate retrieval performance and answer quality.

In [ ]:
def evaluate_generated_response_cosine(generated_response, ground_truth_answer,embedding_model=embedding_model):
    
    """Evaluation for generated response by model vs ground truth answer"""
    generated_response_embeddings = embedding_model.encode_documents([generated_response])['dense'][0]
    ground_truth_answer_embeddings = embedding_model.encode_documents([ground_truth_answer])['dense'][0]

    cosine_score_raw = util.pytorch_cos_sim(generated_response_embeddings, ground_truth_answer_embeddings)
    
    cosine_score = round(float(cosine_score_raw[0][0]) * 100, 2)
    print("Cosine Similarity between generated response and ground truth answer:", cosine_score)
    
    return cosine_score


def evaluate_generated_response_prf(generated_response, ground_truth_answer):
    
    """Evaluation for generated response by model vs ground truth answer"""

    P_raw, R_raw, F1_raw = score([generated_response], [ground_truth_answer], lang='en') # model_type='distilbert-base-uncased'
    P = round(float(P_raw[0]) * 100, 2)
    R = round(float(R_raw[0]) * 100, 2)
    F1 = round(float(F1_raw[0]) * 100, 2)
    print("Precision: ", P)
    print("Recall: ", R)
    print("F1 Score: ", F1)
    
    return P, R, F1

In [ ]:
def rag_chat_with_processing_time(query):
    """RAG chat + processing time """    
    print(f"User query: {query}")

    # Time the response
    start = time.time()
    response, context = rag_chat(query,stream=False)
    end = time.time()
    
    processing_time = f"{end - start:.2f}"
    print(f"\nProcessing time: {processing_time} seconds")
    return response, context, processing_time

In [ ]:
def generate_rag_system_report(questions=questions,answers=answers):
    reports = {
        'question': [],
        'response': [],
        'answer': [],
        'cosine': [],
        'precision': [],
        'recall': [],
        'f1_score': [],
        'context': [],
        'processing_time': []
    }
    for question,answer in zip(questions,answers):
        response, context, processing_time = rag_chat_with_processing_time(question)
        cosine = evaluate_generated_response_cosine(response, answer,embedding_model=embedding_model)
        precision, recall, f1_score = evaluate_generated_response_prf(response, answer)
        reports['question'].append(question)
        reports['response'].append(response)
        reports['answer'].append(answer)
        reports['cosine'].append(cosine)
        reports['precision'].append(precision)
        reports['recall'].append(recall)
        reports['f1_score'].append(f1_score)
        reports['context'].append(context)
        reports['processing_time'].append(processing_time)
        print("======================================")
        print("======================================")
    return reports

reports = generate_rag_system_report()

In [ ]:
def save_rag_system_report(reports):
    df_reports = pd.DataFrame(reports)
    df_reports.to_csv('rag_system_report.csv', index=False)
    print("Cosine Similarity Average:",df_reports['cosine'].mean(), '%')
    print("Sum processing time:",df_reports['processing_time'].astype(float).sum(), 'sec')
    print("Report saved to the file successfully.")
    return df_reports

save_rag_system_report(reports)